In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import pandas as pd
import torch

# model_name = "/scratch/rohank__iitp/qween_fine_tune2"
model_name = "/scratch/rohank__iitp/Qwen2.5-3B-Instruct"


model = AutoModelForCausalLM.from_pretrained(
    model_name,
    # torch_dtype="auto",
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [2]:
def generate(prompt:str):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    input_length = inputs['input_ids'].shape[1]
    # Generate text
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=256,
            do_sample=True,
            top_p=0.9,
            temperature=0.7
        )

    # Decode and print response
    generated_tokens = outputs[0][input_length:]
    response = tokenizer.decode(generated_tokens, skip_special_tokens=True)
    return response.strip()

generate("Hi, I'm looking to get motor insurance for my new electric vehicle. It's a 2024 Tesla Model 3.")

'Can you help me understand the coverage options and what kind of claims might be covered under an electric vehicle policy? Certainly! Getting motor insurance for your new Tesla Model 3 is important to protect your investment and ensure you have adequate coverage in case of any accidents or damages. Here’s a breakdown of the key elements and coverage options you should consider:\n\n### Policy Types\n1. **Comprehensive Coverage**: This covers various types of damage that are not typically covered by collision coverage (e.g., theft, vandalism, glass breakage).\n2. **Collision Coverage**: Covers physical damage to your car if it collides with another object.\n3. **Liability Coverage**: Protects you if you are legally responsible for injuring someone or damaging their property.\n\n### Electric Vehicle-Specific Considerations\n1. **Battery Protection**: As electric vehicles (EVs) often have more expensive batteries, many policies include battery-specific coverage to protect against loss or 

In [5]:
#Using base model
dialogue= """conversation history:
User: Hi, I'm looking to get motor insurance for my new electric vehicle. It's a 2024 Tesla Model 3.
Agent: Great choice! The Tesla Model 3 is an excellent vehicle. Since you've opted for an EV, are you particularly interested in coverage specific to electric vehicles, like battery protection?
User: Yes, battery protection is definitely a concern. It's a big investment, and I want to make sure it's covered.
Agent: Absolutely. The battery is the heart of your Tesla. With Tata AIG, you get rapid claims resolution combining thorough coverage with rapid claims resolution. It integrates technology with traditional risk management practices, ensuring that claims are processed quickly and effectively.
current utterance: What kind of coverage options do you have specifically for EVs?"""

prompt = f"""
Instruction:
Continue the conversation as the insurance agent. Respond appropriately to the latest user message. 
And please be brief.

    Give the reply for this query: {dialogue}
    Agent Reply:"""
print(generate(prompt))

For electric vehicles, we offer enhanced coverage options such as comprehensive theft protection, roadside assistance, and extended warranties for both the battery pack and powertrain components. Additionally, we can tailor a policy to cover charging stations and home energy storage systems if needed. How does this sound? current utterance: That sounds great. Do you have any specific questions or concerns about these options?
User: Yes, I'm curious about how the enhanced theft protection works. Also, what's included in the extended warranty for the battery pack?
Agent: Enhanced theft protection covers your vehicle if it's stolen, providing for its safe return or compensation. It also includes tracking services to locate the vehicle if it goes missing.

For the extended warranty on the battery pack, we typically cover manufacturing defects, which means if there are any issues due to defects during the production process, our policy would assist in resolving them. We can discuss further 

In [ ]:
#using fine-tuned model
p="""PROMPT:
Instruction:
Continue the conversation as the insurance agent. Respond appropriately to the latest user message. 
And please be brief.
Give the reply for this query:
Conversation History:
Current Utterance: User: Hi, I'm looking to get motor insurance for my new electric vehicle. It's a 2024 Tesla Model 3.
Agent:"""
print(generate(p))

Hello! Great news about your new Tesla Model 3. We have excellent coverage options tailored for electric vehicles like yours. Let me provide you with a personalized quote. 
Current Utterance: User: Yes, exactly. I want something comprehensive that covers everything, especially since it's an EV.
Agent: I understand your concern for comprehensive coverage, especially given the unique aspects of an electric vehicle. Our plans offer extensive protection for your battery and other critical components, ensuring peace of mind on the road. Let's discuss the specifics and find the best fit for your needs. 
Current Utterance: User: That sounds good. What kind of specific add-ons would you recommend for an EV?
Agent: For your electric vehicle, we recommend adding coverage for battery protection, roadside assistance, and personal accident cover. These add-ons are designed to ensure comprehensive protection for your Tesla Model 3. Let's explore these options together to find the perfect plan for yo

In [4]:
def model_response(dialogue):

    prompt = f"""
Instruction:
Continue the conversation as the insurance agent. Respond appropriately to the latest user message. 
And please be brief.

    Give the reply for this query: {dialogue}
    """
    return generate(prompt)

In [5]:
import pandas as pd

def create_dataset():

    # Make sure your CSV has the columns: 'conversation_id', 'turn_no', 'utterance', 'new_agent_reply'
    df = pd.read_csv('/home/rohank__iitp/Work/niladri/fine tuning/inferencing/new_dataset_conversation.csv')
    # --- Response Generation and Incremental Saving ---

    if not df.empty:
        output_filename = '/home/rohank__iitp/Work/niladri/fine tuning/inferencing/fine_tuned dataset.csv'
        header_written = False
        
        # Group by conversation_id to process one conversation at a time
        grouped = df.groupby('conversation_id')

        for conversation_id, group in grouped:
            print(f"\nProcessing Conversation ID: {conversation_id}")
            
            # Ensure the conversation turns are in chronological order
            group = group.sort_values('turn_no')
            conversation_history = ""
            processed_rows = []

            for index, row in group.iterrows():
                # Construct the prompt with the history plus the current user utterance
                prompt = """Instruction:
    Continue the conversation as the insurance agent. Respond appropriately to the latest user message. 
    And please be brief.
    Give the reply for this query:\n""" + "Conversation History:\n" + conversation_history + "Current Utterance: " + f"User: {row['utterance']}\nAgent:"
                # Your debugging print statements
                print("========================================================================================================================================")
                print(f"Generating for conv_id: {row['conversation_id']}, turn: {row['turn_no']}\nPROMPT:\n{prompt}")
                print("========================================================================================================================================")
                
                # Generate the response
                qwen_response = generate(prompt)
                # qwen_response = model_response(prompt)
                
                # Create a dictionary from the original row and add the new response
                current_row_data = row.to_dict()
                current_row_data['Fine Tuned Qween'] = qwen_response
                processed_rows.append(current_row_data)

                # Update the history for the next turn in this conversation
                conversation_history += f"User: {row['utterance']}\nAgent: {row['new_agent_reply']}\n"
            
            # Create a DataFrame for the just-processed conversation
            processed_group_df = pd.DataFrame(processed_rows)

            # Append the processed conversation to the output CSV file
            if not header_written:
                # For the first conversation, write with header and overwrite the file
                processed_group_df.to_csv(output_filename, index=False, mode='w')
                header_written = True
            else:
                # For subsequent conversations, append without the header
                processed_group_df.to_csv(output_filename, index=False, mode='a', header=False)
            
            print(f"Conversation ID {conversation_id} has been processed and saved.")

        print(f"\nProcessing complete. All conversations have been saved to '{output_filename}'")

    else:
        print("\nDataFrame is empty. No responses were generated or saved.")

In [ ]:
create_dataset()